# ETL Processes
I use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# creating connection to the database and the cursor 
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
# this function helps me in accessing the files the data is stored in
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files



# Processing `song_data`
In this first part, I perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

I perform ETL on a single song file and load a single record into each table to start.


In [4]:
# Getting the files
song_files = get_files("data/song_data")
song_files

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpoint.json',
 '/home/workspace/data/song_data/A/A/B/TRAABNV128F425CEE1.json',
 '/home/workspace/data/song_data/A/A/B/TRAABRB128F9306DD5.json',
 '/home/workspace/data/song_data/A/A/B/TRAABLR128F423B7E3.js

In [5]:
# choosing one file
filepath = song_files[0]

In [6]:
# accessing the data in the file
import json
with open(filepath) as json_file:
    data = json.load(json_file)
df = pd.DataFrame(data = data, index = [0])
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extracting Data for Songs Table
- I select columns for song ID, title, artist ID, year, and duration
- I index to select the first (only) record in the dataframe and convert the array to a list and set it to `song_data`

In [7]:
song_data =df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()

song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Inserting Record into Song Table
I implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. 

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

## #2: `artists` Table
#### Extracting Data for Artists Table
- I select columns for artist ID, name, location, latitude, and longitude
- I index to select the first (only) record in the dataframe then convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

#### Inserting Record into Artist Table
I implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. 

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

# Processing `log_data`
In this part, I perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

I perform ETL on a single log file and load a single record into each table.
- I use the `get_files` function above to get a list of all log JSON files in `data/log_data`
- I select the first log file in this list and read the log file and view the data

In [11]:
# accessing files
log_files = get_files("data/log_data")
log_files

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

In [12]:
# choosing one file
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [13]:
# accessing data stored in the file
with open(filepath) as log_json:
    log_data = pd.read_json(log_json, lines = True)

df = pd.DataFrame(data = log_data)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extracting Data for Time Table
- I filter records by `NextSong` action
- I convert the `ts` timestamp column to datetime then I extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  

In [14]:
df.shape

(388, 18)

In [15]:
df['page'].value_counts()

NextSong     330
Home          33
Downgrade      6
Login          5
Settings       4
Logout         4
Upgrade        3
Help           2
Error          1
Name: page, dtype: int64

In [16]:
# filtering records by 'Next Song'
for index, value in enumerate(df['page']):
    if value != 'NextSong':
        df.drop(index = index, inplace = True)

df.shape

(330, 18)

In [17]:
# creating 'datetime' column in the dataset
from datetime import datetime
dtime = []
for ts in df['ts']:
    dt = datetime.fromtimestamp(ts/1000)
    dtime.append(dt)
    
df['datetime'] = dtime
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91,2018-11-30 00:22:07.796
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:08:41.796
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:12:48.796
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:17:05.796
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:20:56.796


In [18]:
# getting 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday' from 'datetime'
hour = df['datetime'].dt.hour
day = df['datetime'].dt.day
week_of_year = df['datetime'].dt.weekofyear
month = df['datetime'].dt.month
year = df['datetime'].dt.year
weekday = df['datetime'].dt.weekday







In [19]:
column_labels = ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year','weekday' ]

time_dict = {column_labels[0]:df['ts'], column_labels[1]:hour, column_labels[2]:day, column_labels[3]:week_of_year, column_labels[4]:month, column_labels[5]:year, column_labels[6]: weekday}



In [20]:
time_df = pd.DataFrame(data = time_dict)
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1543537327796,0,30,48,11,2018,4
1,1543540121796,1,30,48,11,2018,4
2,1543540368796,1,30,48,11,2018,4
3,1543540625796,1,30,48,11,2018,4
4,1543540856796,1,30,48,11,2018,4


#### Inserting Records into Time Table
I implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. 

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

## #4: `users` Table
#### Extracting Data for Users Table
- I select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = pd.DataFrame({'user_id': df['userId'], 'first_name':df['firstName'], 'last_name':df['lastName'], 'gender': df['gender'], 'level': df['level']})
user_df.head()

,user_id,first_name,last_name,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


In [23]:
user_df.shape

(330, 5)

#### Inserting Records into Users Table
I implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. 

In [24]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## #5: `songplays` Table
#### Extracting Data and Songplays Table
This task is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, I need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- I implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- I Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`



In [28]:
for index, row in df.iterrows():

    # getting songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # inserting songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

# Close Connection to Sparkify Database

In [29]:
conn.close()